# 패키지 불러오기

In [1]:
'''
<기본적인 전처리>
(1) 종속변수 할당
(2) 물가지수 반영
(3) FA shift + FA 1차, 2차, 3차기간 생성
(4) FA 분리하여 모델링 --> 분리해서 진행하자.
(5) 누적변수 5년, 0.9 ([시간~동시]) --> 4가지 경우의수는 해보자!
***원핫인코딩은 진행

<2. FA/비FA 분류>
<3. Train/test 분류>
'''
import warnings
import math
import time

import pandas as pd
import numpy as np

warnings.filterwarnings(action='ignore')

# 전처리 전체 Flow

### 1. 타자
#### 1. 기본데이터 전처리
 - "생년월일" --> 총 4개(년, 월, 일, 나이) 파생변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - "볼넷/삼진", "땅볼/뜬공" --> 변수제거
 - "연봉" 변수 내 ','를 제거
 - 원핫인코딩 모두 진행 (포지션(수비), 팀명)
 
#### 2. 물가상승률을 반영하여 연봉 변화
 - 한국은행(CPI)기반 2020년으로 연봉가치 통일

#### 3. 종속변수 할당
 - ex) 2016년도 성적을 2017년도 연봉이랑 매칭
 
#### 4. FA Shift / FA 1차, 2차, 3차 변수 생성

#### 5. 누적변수생성 --> (5년, 0.9)는 고정, 시간~동시는 모두 실험

#### 6. FA/비FA 분할
- 분할 후 "ID", "선수명" 변수는 제거
 
#### 7. train/test 데이터 분할
 - test: 2020년 데이터
 - train: 그 외 데이터 모두
 - train/test 분할 후 각각에 대해 결측치가 1개라도 있는 행은 제거

### 2. 투수
#### 1. 기본데이터 전처리
 - "생년월일" --> 총 4개(년, 월, 일, 나이) 파생변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - "이닝수" 0인 선수는 제거
 - "승률" --> 0승 0패는 0으로 대체
 - "삼진/볼넷", "땅볼/뜬공" --> 변수제거
 - "포지션(수비)" 변수는 삭제
 - "연봉" 변수 내 ','를 제거
 - 원핫인코딩 (팀명)

#### 2. 물가상승률을 반영하여 연봉 변화
 - 한국은행(CPI)기반 2020년으로 연봉가치 통일

#### 3. 종속변수 할당
 - ex) 2016년도 성적을 2017년도 연봉이랑 매칭
 
#### 4. FA Shift / FA 1차, 2차, 3차 변수 생성

#### 5. 누적변수생성 --> (5년, 0.9)는 고정, 시간~동시는 모두 실험

#### 6. FA/비FA 분할
- 분할 후 "ID", "선수명" 변수는 제거
 
#### 7. train/test 데이터 분할
 - test: 2020년 데이터
 - train: 그 외 데이터 모두
 - train/test 분할 후 각각에 대해 결측치가 1개라도 있는 행은 제거

# 1. 기본 전처리

In [2]:
# 연봉 및 수상횟수 전처리

def salary_preprocessing(value):
    try:
        value = value.replace(',', '')
    
    except:
        pass
    
    return value

#### (1) 타자

In [3]:
# 데이터 불러오기
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')

# 생년월일 전처리
hitter['출생연도'] = hitter['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
hitter['출생월'] = hitter['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
hitter['출생일'] = hitter['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
hitter['나이'] = hitter['연도'] - hitter['출생연도'] + 1
hitter = hitter.drop('생년월일', axis=1)

# 볼넷/삼진, 땅볼/뜬공 --> 제거
hitter = hitter.drop(['볼넷/삼진', '땅볼/뜬공'], axis=1)

# 연봉 내 ','를 제거
hitter['연봉'] = list(map(lambda x: salary_preprocessing(x), hitter['연봉']))

# 원핫인코딩
hitter = pd.get_dummies(hitter, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')
hitter = pd.get_dummies(hitter, columns=['포지션(수비)'], prefix='포지션(수비)', drop_first=False, dtype='int')

# 데이터 형변환
hitter['데뷔년도'] = hitter['데뷔년도'].astype('int')
hitter['타율'] = hitter['타율'].replace('-', 0).astype('float')
hitter['장타율'] = hitter['장타율'].replace('-', 0).astype('float')
hitter['순수장타율'] = hitter['순수장타율'].replace('-', 0).astype('float')
hitter['연봉'] = hitter['연봉'].astype('float')
hitter['수상횟수'] = hitter['수상횟수'].astype('int')
hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']] = hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']].astype('int')

#### (2) 투수

In [4]:
# 데이터 불러오기
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')

# 생년월일 전처리
pitcher['출생연도'] = pitcher['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
pitcher['출생월'] = pitcher['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
pitcher['출생일'] = pitcher['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
pitcher['나이'] = pitcher['연도'] - pitcher['출생연도'] + 1
pitcher = pitcher.drop('생년월일', axis=1)

# 이닝수가 1/3이상인 선수만 분석
pitcher = pitcher[pitcher['이닝'] != 0].reset_index(drop=True)

# 승률 0승0패 --> 0
pitcher['승률'] = pitcher['승률'].replace('-',0).astype('float')

# 삼진/볼넷, 땅볼/뜬공 --> 변수제거
pitcher = pitcher.drop(['삼진/볼넷', '땅볼/뜬공'], axis=1)

# 포지션(수비) 변수 삭제
pitcher = pitcher.drop('포지션(수비)', axis=1)

# 연봉 데이터 전처리
pitcher['연봉'] = list(map(lambda x: salary_preprocessing(x), pitcher['연봉']))

# 원핫인코딩
pitcher = pd.get_dummies(pitcher, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')

# 데이터 형변환
pitcher['데뷔년도'] = pitcher['데뷔년도'].astype('int')
pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']] = pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']].astype('float')
pitcher['연봉'] = pitcher['연봉'].astype('float')
pitcher['수상횟수'] = pitcher['수상횟수'].astype('int')

# 2. 물가상승률(CPI)반영

#### (1) 물가상승률 계산

In [5]:
money_rate = pd.read_excel('물가상승률데이터/소비자_물가지수(한국은행).xlsx')
money_rate = money_rate.loc[:, '2002':].transpose()
money_rate = money_rate.reset_index()
money_rate.columns = ['연도','소비자물가지수']

for idx in range(money_rate.shape[0]):
    money_rate.loc[idx, '2020년기준상수'] = money_rate.loc[money_rate.shape[0]-1, '소비자물가지수'] / money_rate.loc[idx, '소비자물가지수']
    
money_rate

,연도,소비자물가지수,2020년기준상수
0,2002,71.193,1.480764
1,2003,73.695,1.430491
2,2004,76.341,1.380909
3,2005,78.444,1.343889
4,2006,80.202,1.314431
5,2007,82.235,1.281936
6,2008,86.079,1.224689
7,2009,88.452,1.191833
8,2010,91.051,1.157813
9,2011,94.717,1.113000


#### (2) 연봉 변환

In [6]:
for i in range(hitter.shape[0]):
    year = hitter.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == f'{year}', '2020년기준상수'])
    hitter.loc[i, '연봉'] = hitter.loc[i, '연봉'] * interest_rate


for i in range(pitcher.shape[0]):
    year = pitcher.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == f'{year}', '2020년기준상수'])
    pitcher.loc[i, '연봉'] = pitcher.loc[i, '연봉'] * interest_rate

# 3. 종속변수 할당

#### (1) 타자

In [7]:
hitter_final = pd.DataFrame()

for ID in hitter['ID'].unique():

    tmp = hitter.loc[hitter['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    hitter_final = pd.concat([hitter_final, tmp]).reset_index(drop=True)

#### (2) 투수

In [8]:
pitcher_final = pd.DataFrame()

for ID in pitcher['ID'].unique():

    tmp = pitcher.loc[pitcher['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    pitcher_final = pd.concat([pitcher_final, tmp]).reset_index(drop=True)

# 4. FA 1차, 2차, 3차 변수 생성
 - 이때, 주의사항이 있는데, FA데이터를 나눌 때 2021년에 신규 FA가 되는 선수의 경우도 FA에 나눠주자.
 - 이 이유는, 결국 우리가 Shift를 진행하게 될텐데, 현재 데이터상에는 2020년까지만 수집했기에, 2021년은 반영되지 않았다.
 - 이를 인터넷에서 직접 파악하였고, 수기로 추가한다.

#### (1) FA데이터 불러오기

In [9]:
fa_data = pd.read_csv('FA정보데이터/FA_정보.csv')
fa_data = fa_data[~((fa_data['선수명'] == '이승호') & (fa_data['생년월일'] == '1976-08-23'))].reset_index(drop=True) # 불필요한 데이터 삭제: 해당 이승호는 pitcher 데이터프레임에 없기에, merge시 다른 이승호에 겹쳐 2개가 생기는 경우가 발생
fa_data = fa_data[['선수명','연도','계약기간']]
fa_data = fa_data.dropna().reset_index(drop=True)
fa_data['계약기간'] = fa_data['계약기간'].apply(lambda x: int(x.replace('년','')))
fa_data                  

,선수명,연도,계약기간
0,오주원,2020,2
1,김태균,2020,1
2,오재원,2020,3
3,김태군,2020,4
4,이성열,2020,2
...,...,...,...
225,안경현,2003,4
226,양준혁,2002,4
227,김민재,2002,4
228,전준호,2002,3


In [10]:
'''
2021년 FA계약자 16
유희관 이용찬 김재호 오재일 최주환 허경민 정수빈
차우찬 김용의
김상수
최형우 양현종(해외)
이대호
우규민 이원석
김성현
'''


fa2021_hitter = ['김재호', '오재일', '최주환', '허경민', '정수빈', '김용의', '최형우', '이대호', '이원석', '김성현'] # 이원석만 동명이인 존재 --> 68700이원석은 삭제 [74206, 75334, 76267, 79240, 79231, 78217, 72443, 71564, 75566, 76802]
fa2021_pitcher = ['유희관', '이용찬', '차우찬', '김상수', '양현종', '우규민'] # 동명이인X

hitter_2021_IDs = []
for name in fa2021_hitter:
    hitter_2021_IDs.extend(list(hitter.loc[hitter['선수명'] == name, 'ID'].unique()))
    
hitter_2021_IDs.remove(68700)

pitcher_2021_IDs = []
for name in fa2021_pitcher:
    pitcher_2021_IDs.extend(list(pitcher.loc[pitcher['선수명'] == name, 'ID'].unique()))

#### (2) 타자

In [11]:
# fa, 비fa 파악 후 2021년 fa데이터를 fa_ids에 추가
fa_ids = list(hitter_final.loc[hitter_final['FA여부'] == 1, 'ID'].unique())
fa_ids.extend(hitter_2021_IDs)
nonfa_hitter_ids = set(hitter_final['ID']) - set(fa_ids)

# 타자 계약기간 data merge 
hitter_final_fa = hitter_final[hitter_final['ID'].isin(fa_ids)].sort_values(by=['ID','연도']).reset_index(drop=True)
hitter_final_fa = pd.merge(hitter_final_fa, fa_data, how='left', on=['선수명','연도'])

# merge 예외사항 후처리 (김태균(71752)은 동일년도, 동일이름이 존재하는 유일한 경우로, 해당하지 않는 경우인 김태균은 FA로 처리X)
hitter_final_fa.loc[(hitter_final_fa['ID'] == 71752) & (hitter_final_fa['연도'] == 2005), '계약기간'] = np.nan

In [12]:
# 타자: 1차, 2차, 3차 파생변수 생성
hitter_final_fa2 = pd.DataFrame()
for ID in hitter_final_fa['ID'].unique():
    
    tmp = hitter_final_fa[hitter_final_fa['ID'] == ID].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])

    for idx, year in enumerate(years):

        contract_years = [year+i for i in range(int(tmp.loc[tmp['연도'] == year, '계약기간']))]

        if idx == 0:
            tmp.loc[tmp['연도'].isin(contract_years), '1차FA여부'] = 1
        elif idx == 1:
            tmp.loc[tmp['연도'].isin(contract_years), '2차FA여부'] = 1
        elif idx == 2:
            tmp.loc[tmp['연도'].isin(contract_years), '3차FA여부'] = 1 
        else:
            print(f'{ID}에 4차 이상의 FA존재')
            
    hitter_final_fa2 = pd.concat([hitter_final_fa2, tmp]).reset_index(drop=True)

In [13]:
# 계약기간 변수 삭제
hitter_final_fa2 = hitter_final_fa2.drop('계약기간', axis=1)

# 파생변수 null값에 0으로 채우기
hitter_final_fa2['1차FA여부'] = hitter_final_fa2['1차FA여부'].fillna(0)
hitter_final_fa2['2차FA여부'] = hitter_final_fa2['2차FA여부'].fillna(0)
hitter_final_fa2['3차FA여부'] = hitter_final_fa2['3차FA여부'].fillna(0)

#### (3) 투수

In [14]:
# fa, 비fa 파악 후 2021년 fa데이터를 fa_ids에 추가
fa_ids = list(pitcher_final.loc[pitcher_final['FA여부'] == 1, 'ID'].unique())
fa_ids.extend(pitcher_2021_IDs)
nonfa_pitcher_ids = set(pitcher_final['ID'])- set(fa_ids)

# 투수 계약기간 data merge 
pitcher_final_fa = pitcher_final[pitcher_final['ID'].isin(fa_ids)].sort_values(by=['ID','연도']).reset_index(drop=True)
pitcher_final_fa = pd.merge(pitcher_final_fa, fa_data, how='left', on=['선수명','연도'])

In [15]:
# 투수: 1차, 2차, 3차 파생변수 생성
pitcher_final_fa2 = pd.DataFrame()
for ID in pitcher_final_fa['ID'].unique():
    
    tmp = pitcher_final_fa[pitcher_final_fa['ID'] == ID].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])

    for idx, year in enumerate(years):

        contract_years = [year+i for i in range(int(tmp.loc[tmp['연도'] == year, '계약기간']))]

        if idx == 0:
            tmp.loc[tmp['연도'].isin(contract_years), '1차FA여부'] = 1
        elif idx == 1:
            tmp.loc[tmp['연도'].isin(contract_years), '2차FA여부'] = 1
        elif idx == 2:
            tmp.loc[tmp['연도'].isin(contract_years), '3차FA여부'] = 1 
        else:
            print(f'{ID}에 4차 이상의 FA존재')
            
    pitcher_final_fa2 = pd.concat([pitcher_final_fa2, tmp]).reset_index(drop=True)

In [16]:
# 계약기간 변수 삭제
pitcher_final_fa2 = pitcher_final_fa2.drop('계약기간', axis=1)

# 파생변수 null값에 0으로 채우기
pitcher_final_fa2['1차FA여부'] = pitcher_final_fa2['1차FA여부'].fillna(0)
pitcher_final_fa2['2차FA여부'] = pitcher_final_fa2['2차FA여부'].fillna(0)

# 5. FA, FA1~3차 데이터 Shift

#### (1) 타자

In [17]:
hitter_final_fa3 = pd.DataFrame()

for ID in hitter_final_fa2['ID'].unique():
    
    tmp = hitter_final_fa2[hitter_final_fa2['ID'] == ID].reset_index(drop=True)
    
    tmp['FA여부'] = tmp['FA여부'].shift(-1)
    tmp['1차FA여부'] = tmp['1차FA여부'].shift(-1)
    tmp['2차FA여부'] = tmp['2차FA여부'].shift(-1)
    tmp['3차FA여부'] = tmp['3차FA여부'].shift(-1)
    
    hitter_final_fa3 = pd.concat([hitter_final_fa3, tmp]).reset_index(drop=True)

#### (2) 투수

In [18]:
pitcher_final_fa3 = pd.DataFrame()

for ID in pitcher_final_fa2['ID'].unique():
    
    tmp = pitcher_final_fa2[pitcher_final_fa2['ID'] == ID].reset_index(drop=True)
    
    tmp['FA여부'] = tmp['FA여부'].shift(-1)
    tmp['1차FA여부'] = tmp['1차FA여부'].shift(-1)
    tmp['2차FA여부'] = tmp['2차FA여부'].shift(-1)
    
    pitcher_final_fa3 = pd.concat([pitcher_final_fa3, tmp]).reset_index(drop=True)

#### (3) 해외파 FA오류 수정
 - 김현수: 2차FA여부 --> [2018, 2019도 1년여부 -> 2년여부로 수정]
 - 이대호: 3차FA여부 FA여부  --> [2017~2019 1년여부 -> 2년여부로 수정]

In [19]:
hitter_final_fa3.loc[(hitter_final_fa3['선수명'] == '김현수') & (hitter_final_fa3['연도'].isin([2018, 2019])), '1차FA여부'] = 0
hitter_final_fa3.loc[(hitter_final_fa3['선수명'] == '김현수') & (hitter_final_fa3['연도'].isin([2018, 2019])), '2차FA여부'] = 1
hitter_final_fa3.loc[(hitter_final_fa3['선수명'] == '이대호') & (hitter_final_fa3['연도'].isin([2017, 2018, 2019])), '1차FA여부'] = 0
hitter_final_fa3.loc[(hitter_final_fa3['선수명'] == '이대호') & (hitter_final_fa3['연도'].isin([2017, 2018, 2019])), '2차FA여부'] = 1

#### (3) 기존 fa선수 파악 및 신규 fa선수를 파악하여 2020년 데이터의 FA여부 데이터 보간
 - 2020년에 FA여부를 찾아야하는 선수들 중 동명이인은 존재하지 않음

In [20]:
# fa인 애들 식별: 기존 fa_data에 + 2021년에 fa계약한 애들을 합쳐서 생각

# 기존데이터: fa계약연도와 계약기간을 고려하여 2020년에 계약한 사람들을 파악
fa_data2 = fa_data.copy()
fa_data2['연도'] = fa_data2['연도'] - 1
result = pd.DataFrame()

for name, year, length in zip(fa_data2['선수명'], fa_data2['연도'], fa_data2['계약기간']):
    contract_years = [year + i for i in range(length)]
    
    for contract_year in contract_years:
        
        tmp = pd.DataFrame([name, contract_year]).T
        tmp.columns = ['선수명','연도']
        
        result = pd.concat([result, tmp])
        
fa2020_players = result.loc[result['연도'] == 2020, '선수명'].unique()


# fa2020_players기준으로 타자/투수여부 수기 분류
fa2020_hitter = ['오재원', '김태군', '이성열', '김선빈', '김강민', '전준우', '박석민', '안치홍', '오지환', '유한준', '이지영', '김민성',
                '김상수', '박경수', '양의지', '이재원', '최정', '모창민', '김현수', '정의윤', '민병헌', '손아섭', '강민호', '황재균']
fa2020_pitcher = ['오주원', '윤규진', '진해수', '송은범', '정우람', '노경은', '이보근']

fa2020_hitter_concat = set(fa2020_hitter + fa2021_hitter)
fa2020_pitcher_concat = set(fa2020_pitcher + fa2021_pitcher)

In [21]:
# 타자 FA여부 보간
hitter_final_fa3.loc[(hitter_final_fa3['연도'] == 2020), 'FA여부'] = 0
hitter_final_fa3.loc[(hitter_final_fa3['연도'] == 2020) & (hitter_final_fa3['선수명'].isin(fa2021_hitter)), 'FA여부'] = 1

# 투수 FA여부 보간
pitcher_final_fa3.loc[(pitcher_final_fa3['연도'] == 2020), 'FA여부'] = 0
pitcher_final_fa3.loc[(pitcher_final_fa3['연도'] == 2020) & (pitcher_final_fa3['선수명'].isin(fa2021_pitcher)), 'FA여부'] = 1

In [22]:
# 타자: 1~3차 FA여부 보간
'''
강민호: 2차FA여부
김강민: 2차FA여부
김민성: 1차FA여부
김상수: 1차FA여부
김선빈: 1차FA여부
김성현: 1차FA여부 FA여부
김용의: 1차FA여부 FA여부
김재호: 2차FA여부 FA여부
김태군: 1차FA여부
김현수: 2차FA여부
모창민: 1차FA여부
민병헌: 1차FA여부
박경수: 2차FA여부
박석민: 2차FA여부
손아섭: 1차FA여부
안치홍: 1차FA여부
양의지: 1차FA여부
오재원: 2차FA여부
오재일: 1차FA여부 FA여부
오지환: 1차FA여부
유한준: 2차FA여부
이대호: 3차FA여부 FA여부
이성열: 2차FA여부
이원석: 2차FA여부 FA여부
이재원: 1차FA여부
이지영: 1차FA여부
전준우: 1차FA여부
정수빈: 1차FA여부 FA여부
정의윤: 1차FA여부
최정: 2차FA여부
최주환: 1차FA여부 FA여부
최형우: 2차FA여부 FA여부
허경민: 1차FA여부 FA여부
황재균: 1차FA여부
'''

fa_1_period = ['김민성', '김상수', '김선빈', '김성현', '김용의', '김태군', '모창민', '민병헌', '손아섭', '안치홍', '양의지', 
              '오재일', '오지환', '이재원', '이지영', '전준우', '정수빈', '정의윤', '최주환', '허경민', '황재균']
fa_2_period = ['강민호', '김강민', '김재호', '김현수', '박경수', '박석민', '오재원', '유한준', '이성열', '이원석', '최정',
              '최형우']
fa_3_period = ['이대호']

hitter_final_fa3.loc[hitter_final_fa3['연도'] == 2020, '1차FA여부'] = 0
hitter_final_fa3.loc[hitter_final_fa3['연도'] == 2020, '2차FA여부'] = 0
hitter_final_fa3.loc[hitter_final_fa3['연도'] == 2020, '3차FA여부'] = 0

hitter_final_fa3.loc[(hitter_final_fa3['연도'] == 2020) & (hitter_final_fa3['선수명'].isin(fa_1_period)), '1차FA여부'] = 1
hitter_final_fa3.loc[(hitter_final_fa3['연도'] == 2020) & (hitter_final_fa3['선수명'].isin(fa_2_period)), '2차FA여부'] = 1
hitter_final_fa3.loc[(hitter_final_fa3['연도'] == 2020) & (hitter_final_fa3['선수명'].isin(fa_3_period)), '3차FA여부'] = 1

In [23]:
# 투수: 1~2차FA여부 보간
'''
2020년
김상수: 1차FA여부 FA여부
노경은: 1차FA여부 
송은범: 2차FA여부
양현종: 2차FA여부 FA여부
오주원: 1차FA여부
우규민: 2차FA여부 FA여부
유희관: 1차FA여부 FA여부
윤규진: 1차FA여부
이보근: 1차FA여부
이용찬: 1차FA여부 FA여부
정우람: 2차FA여부
진해수: 1차FA여부
차우찬: 2차FA여부 FA여부
'''

fa_1_period = ['김상수', '노경은', '오주원', '유희관', '윤규진', '이보근', '이용찬', '진해수']
fa_2_period = ['송은범', '양현종', '우규민', '정우람', '차우찬']

pitcher_final_fa3.loc[pitcher_final_fa3['연도'] == 2020, '1차FA여부'] = 0
pitcher_final_fa3.loc[pitcher_final_fa3['연도'] == 2020, '2차FA여부'] = 0

pitcher_final_fa3.loc[(pitcher_final_fa3['연도'] == 2020) & (pitcher_final_fa3['선수명'].isin(fa_1_period)), '1차FA여부'] = 1
pitcher_final_fa3.loc[(pitcher_final_fa3['연도'] == 2020) & (pitcher_final_fa3['선수명'].isin(fa_2_period)), '2차FA여부'] = 1

# 6. fa와 비fa 데이터 병합 후 결측치 컨트롤
 - 결측치 제거를 주의해서 하는 이유는, 합친데이터에서 결측치 제거 시 2020년이 모두 날아가기 때문
 - 이에 대하여 2019년만 날린 후, 다시 병합
 - fa와 비fa합치기 -> 2019이전과 2020나눈 후 2019이전 데이터 결측치 모두 제거 -> 2019이전과 2020년 다시 합치기

#### (1) 타자

In [24]:
# nonfa 데이터 식별
hitter_nonfa = hitter_final[hitter_final['ID'].isin(nonfa_hitter_ids)].reset_index(drop=True)
hitter_nonfa[['1차FA여부', '2차FA여부', '3차FA여부']] = 0

# fa와 비fa 병합
hitter_all = pd.concat([hitter_nonfa, hitter_final_fa3]).reset_index(drop=True)

# 2019년이전 데이터 결측치 제거후 다시 병합
hitter_all_2019 = hitter_all[hitter_all['연도'] <= 2019]
hitter_all_2020 = hitter_all[hitter_all['연도'] == 2020]
hitter_all2 = pd.concat([hitter_all_2019.dropna(), hitter_all_2020]).reset_index(drop=True)

#### (2) 투수
- 2020년에 FA여부를 찾아야하는 선수들 중 동명이인은 존재하지 않음

In [25]:
# nonfa 데이터 식별
pitcher_nonfa = pitcher_final[pitcher_final['ID'].isin(nonfa_pitcher_ids)].reset_index(drop=True)
pitcher_nonfa[['1차FA여부', '2차FA여부']] = 0

# fa와 비fa 병합
pitcher_all = pd.concat([pitcher_nonfa, pitcher_final_fa3]).reset_index(drop=True)

# 2019년이전 데이터 결측치 제거후 다시 병합
pitcher_all_2019 = pitcher_all[pitcher_all['연도'] <= 2019]
pitcher_all_2020 = pitcher_all[pitcher_all['연도'] == 2020]
pitcher_all2 = pd.concat([pitcher_all_2019.dropna(), pitcher_all_2020]).reset_index(drop=True)

# 7. 누적변수 생성

#### (1) 함수정의

In [26]:
# 시간가중치 생성
def make_exponential_weight(length, a):
    return [a**(i-1) for i in range(length, 0, -1)]

##################################################################################################
##################################################################################################

# 단순 평균
def simple_creation(dataset, n_year, feature):

    final_dataset = pd.DataFrame()
    
    # ID별로 접근
    for ID in dataset['ID'].unique():

        tmp = dataset.loc[dataset['ID'] == ID].sort_values(by='연도').reset_index(drop=True)
        
        # 파생변수 생성
        for i in range(tmp.shape[0]):
            
            if feature == '연봉':
                tmp.loc[i, f'{n_year}년평균{feature}'] = tmp.loc[i-n_year:i-1, feature].mean()
            else:
                tmp.loc[i, f'{n_year}년평균{feature}'] = tmp.loc[i-n_year+1:i, feature].mean()
            
        final_dataset = pd.concat([final_dataset, tmp]).reset_index(drop=True)

    return final_dataset

##################################################################################################
##################################################################################################

# 가중시간 평균
def time_weighted_creation(dataset, n_year, feature, alpha):

    final_dataset = pd.DataFrame()
    
    # ID별로 접근
    for ID in dataset['ID'].unique():

        tmp = dataset[dataset['ID'] == ID].sort_values(by='연도').reset_index(drop=True)
        
        # 파생변수 생성
        for i in range(tmp.shape[0]):
            
            if feature == '연봉':
                value = np.array(tmp.loc[i-n_year:i-1, feature])
            else:
                value = np.array(tmp.loc[i-n_year+1:i, feature])
                
            weight = np.array(make_exponential_weight(length = value.shape[0], 
                                                      a = alpha))
            
            tmp.loc[i, f'{n_year}년평균{feature}'] = np.matmul(value, weight) / np.sum(weight)
            
        final_dataset = pd.concat([final_dataset, tmp]).reset_index(drop=True)

    return final_dataset

##################################################################################################
##################################################################################################

# 가중타석(또는 이닝) 평균
def play_weighted_creation(dataset, n_year, feature, criteria):

    final_dataset = pd.DataFrame()
    
    # ID별로 접근
    for ID in dataset['ID'].unique():

        tmp = dataset[dataset['ID'] == ID].sort_values(by='연도').reset_index(drop=True)
        
        # 파생변수 생성
        for i in range(tmp.shape[0]):
            
            if feature =='연봉':
                value = np.array(tmp.loc[i-n_year:i-1, feature])
                weight = np.array(tmp.loc[i-n_year:i-1, criteria])
                
            else:
                value = np.array(tmp.loc[i-n_year+1:i, feature])
                weight = np.array(tmp.loc[i-n_year+1:i, criteria])
                
            tmp.loc[i, f'{n_year}년평균{feature}'] = np.matmul(value, weight) / np.sum(weight)
            
        final_dataset = pd.concat([final_dataset, tmp]).reset_index(drop=True)

    return final_dataset

##################################################################################################
##################################################################################################

# 시간 + 타석(또는 이닝) 평균
def timeandplay_weighted_creation(dataset, n_year, feature, criteria, alpha):

    final_dataset = pd.DataFrame()
    
    # ID별로 접근
    for ID in dataset['ID'].unique():

        tmp = dataset[dataset['ID'] == ID].sort_values(by='연도').reset_index(drop=True)
        
        # 파생변수 생성
        for i in range(tmp.shape[0]):
            
            if feature == '연봉':
                value = np.array(tmp.loc[i-n_year:i-1, feature])
                play_weight = np.array(tmp.loc[i-n_year:i-1, criteria])
                
            else:
                value = np.array(tmp.loc[i-n_year+1:i, feature])
                play_weight = np.array(tmp.loc[i-n_year+1:i, criteria])
                
            time_weight = np.array(make_exponential_weight(length = value.shape[0], 
                                                          a = alpha))
            
            tmp.loc[i, f'{n_year}년평균{feature}'] = np.matmul(np.multiply(value, play_weight), time_weight) / np.matmul(play_weight, time_weight)
                        
        final_dataset = pd.concat([final_dataset, tmp]).reset_index(drop=True)

    return final_dataset

##################################################################################################
##################################################################################################

# 종합 함수
def make_average_variable(dataset, n_year, feature, criteria, alpha, how):
    
    if how == 'simple':
        return simple_creation(dataset, n_year, feature)
    
    elif how == 'time_weighted':
        return time_weighted_creation(dataset, n_year, feature, alpha)
    
    elif how == 'play_weighted':
        return play_weighted_creation(dataset, n_year, feature, criteria)
    
    elif how == 'time_and_play_weighted':
        return timeandplay_weighted_creation(dataset, n_year, feature, criteria, alpha)
    
##################################################################################################
##################################################################################################
    
# N년평균연봉 결측치 대체 함수 (최저연봉대체)

def data_fillna(dataframe, n_year):
    
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2002), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2002, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2003), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2003, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2004), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2004, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2005), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2005, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2006), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2006, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2007), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2007, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2008), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2008, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2009), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2009, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2010), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2010, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2011), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2011, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2012), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2012, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2013), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2013, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2014), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2014, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2015), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2015, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2016), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2016, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2017), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2017, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2018), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2018, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2019), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2019, '최저연봉'])
    dataframe.loc[(dataframe[f'{n_year}년평균연봉'].isnull()) & (dataframe['연도'] == 2020), f'{n_year}년평균연봉'] = float(minimum_money.loc[minimum_money['연도'] == 2020, '최저연봉'])

    return dataframe

#### (2) 누적변수를 만들 변수들 정의

In [27]:
# 타자 변수들 중 누적변수를 생성할 변수 추리기
hitter_columns = list(hitter_all2.columns)
hitter_remove_cols = ['ID', '선수명', '데뷔년도', '연도', 'FA여부', '뉴스개수(누적정규화)', '누적타석', '출생연도', '출생월', '출생일', '나이', 
                  '팀명_KIA', '팀명_KT', '팀명_LG', '팀명_NC', '팀명_SK', '팀명_두산', '팀명_롯데', '팀명_삼성',
                  '팀명_우리/히어로즈/넥센/키움', '팀명_한화', '팀명_현대', '포지션(수비)_1루수', '포지션(수비)_2루수',
                  '포지션(수비)_3루수', '포지션(수비)_수비기록없음', '포지션(수비)_우익수', '포지션(수비)_유격수',
                  '포지션(수비)_좌익수', '포지션(수비)_중견수', '포지션(수비)_포수', '1차FA여부', '2차FA여부', '3차FA여부']

for hitter_remove_col in hitter_remove_cols:
    hitter_columns.remove(hitter_remove_col)
    
    

# 투수 변수들 중 누적변수를 생성할 변수 추리기
pitcher_columns = list(pitcher_all2.columns)
pitcher_remove_cols = ['ID', '선수명', '데뷔년도', '연도', 'FA여부', '뉴스개수(누적정규화)', '누적이닝', '출생연도', '출생월', '출생일', '나이',
                      '팀명_KIA', '팀명_KT', '팀명_LG', '팀명_NC', '팀명_SK', '팀명_두산', '팀명_롯데', '팀명_삼성', 
                      '팀명_우리/히어로즈/넥센/키움', '팀명_한화', '팀명_현대', '1차FA여부', '2차FA여부']

for pitcher_remove_col in pitcher_remove_cols:
    pitcher_columns.remove(pitcher_remove_col)
    

print(f'타자 파생변수 개수: {len(hitter_columns)}')
print(f'투수 파생변수 개수: {len(pitcher_columns)}')

타자 파생변수 개수: 57
투수 파생변수 개수: 61


#### (3) 최저연봉정의: 이는 연봉데이터의 경우, 올해부터가 아닌 작년부터 평균을 연산하기에 첫 해가 문제가 될 수 있다. 이러한 첫 해에는 최저연봉값을 넣어주기 위함이다.

In [28]:
# 최저연봉 데이터프레임 구성

### 최저연봉데이터
money = pd.read_excel('연봉데이터/연도별_최저연봉.xlsx')
money['최저연봉'] = money['최저연봉'].apply(lambda x: int(x.replace(',', '')))

### 소비자물가지수
money_rate = pd.read_excel('물가상승률데이터/소비자_물가지수(한국은행).xlsx')
money_rate = money_rate.loc[:, '2002':].transpose()
money_rate = money_rate.reset_index()
money_rate.columns = ['연도','소비자물가지수']
for idx in range(money_rate.shape[0]):
    money_rate.loc[idx, '2020년기준상수'] = money_rate.loc[money_rate.shape[0]-1, '소비자물가지수'] / money_rate.loc[idx, '소비자물가지수']
    
### 소비자물가지수를 반영한 최저연봉
minimum_money = pd.DataFrame({'연도':list(money['연도']),
                              '최저연봉':list(money['최저연봉'] * money_rate['2020년기준상수'])})
minimum_money

,연도,최저연봉
0,2002,2517.298049
1,2003,2431.833910
2,2004,2485.636814
3,2005,2687.777268
4,2006,2628.862123
5,2007,2563.871831
6,2008,2449.377897
7,2009,2383.665717
8,2010,2778.750371
9,2011,2671.199468


In [29]:
YEAR = 5
ALPHA = 0.9

In [30]:
# simple 방식

t1 = time.time()

################## 타자 ##################
for idx, hitter_column in enumerate(hitter_columns):
    
    tmp = make_average_variable(hitter_all2, YEAR, hitter_column, '타석', ALPHA, 'simple')
    
    if idx == 0:
        hitter_simple = tmp.copy()
    else:
        hitter_simple = pd.concat([hitter_simple, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(hitter_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
        
################## 투수 ##################
for idx, pitcher_column in enumerate(pitcher_columns):
    
    tmp = make_average_variable(pitcher_all2, YEAR, pitcher_column, '이닝', ALPHA, 'simple')
    
    if idx == 0:
        pitcher_simple = tmp.copy()
    else:
        pitcher_simple = pd.concat([pitcher_simple, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(pitcher_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
hitter_simple = data_fillna(hitter_simple, YEAR)
pitcher_simple = data_fillna(pitcher_simple, YEAR)

10/57개 생성 완료!! == 경과시간: 60초
20/57개 생성 완료!! == 경과시간: 119초
30/57개 생성 완료!! == 경과시간: 178초
40/57개 생성 완료!! == 경과시간: 236초
50/57개 생성 완료!! == 경과시간: 294초
10/61개 생성 완료!! == 경과시간: 384초
20/61개 생성 완료!! == 경과시간: 434초
30/61개 생성 완료!! == 경과시간: 484초
40/61개 생성 완료!! == 경과시간: 533초
50/61개 생성 완료!! == 경과시간: 583초
60/61개 생성 완료!! == 경과시간: 633초


In [31]:
# 시간가중 방식

t1 = time.time()

################## 타자 ##################
for idx, hitter_column in enumerate(hitter_columns):
    
    tmp = make_average_variable(hitter_all2, YEAR, hitter_column, '타석', ALPHA, 'time_weighted')
    
    if idx == 0:
        hitter_time_weighted = tmp.copy()
    else:
        hitter_time_weighted = pd.concat([hitter_time_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(hitter_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
        
################## 투수 ##################
for idx, pitcher_column in enumerate(pitcher_columns):
    
    tmp = make_average_variable(pitcher_all2, YEAR, pitcher_column, '이닝', ALPHA, 'time_weighted')
    
    if idx == 0:
        pitcher_time_weighted = tmp.copy()
    else:
        pitcher_time_weighted = pd.concat([pitcher_time_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(pitcher_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
hitter_time_weighted = data_fillna(hitter_time_weighted, YEAR)
pitcher_time_weighted = data_fillna(pitcher_time_weighted, YEAR)

10/57개 생성 완료!! == 경과시간: 60초
20/57개 생성 완료!! == 경과시간: 116초
30/57개 생성 완료!! == 경과시간: 173초
40/57개 생성 완료!! == 경과시간: 231초
50/57개 생성 완료!! == 경과시간: 291초
10/61개 생성 완료!! == 경과시간: 385초
20/61개 생성 완료!! == 경과시간: 436초
30/61개 생성 완료!! == 경과시간: 486초
40/61개 생성 완료!! == 경과시간: 538초
50/61개 생성 완료!! == 경과시간: 586초
60/61개 생성 완료!! == 경과시간: 638초


In [32]:
# 타석(또는 이닝)가중 방식

t1 = time.time()

################## 타자 ##################
for idx, hitter_column in enumerate(hitter_columns):
    
    tmp = make_average_variable(hitter_all2, YEAR, hitter_column, '타석', ALPHA, 'play_weighted')
    
    if idx == 0:
        hitter_play_weighted = tmp.copy()
    else:
        hitter_play_weighted = pd.concat([hitter_play_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(hitter_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
        
################## 투수 ##################
for idx, pitcher_column in enumerate(pitcher_columns):
    
    tmp = make_average_variable(pitcher_all2, YEAR, pitcher_column, '이닝', ALPHA, 'play_weighted')
    
    if idx == 0:
        pitcher_play_weighted = tmp.copy()
    else:
        pitcher_play_weighted = pd.concat([pitcher_play_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(pitcher_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
hitter_play_weighted = data_fillna(hitter_play_weighted, YEAR)
pitcher_play_weighted = data_fillna(pitcher_play_weighted, YEAR)

10/57개 생성 완료!! == 경과시간: 63초
20/57개 생성 완료!! == 경과시간: 124초
30/57개 생성 완료!! == 경과시간: 181초
40/57개 생성 완료!! == 경과시간: 237초
50/57개 생성 완료!! == 경과시간: 294초
10/61개 생성 완료!! == 경과시간: 382초
20/61개 생성 완료!! == 경과시간: 431초
30/61개 생성 완료!! == 경과시간: 480초
40/61개 생성 완료!! == 경과시간: 529초
50/61개 생성 완료!! == 경과시간: 578초
60/61개 생성 완료!! == 경과시간: 629초


In [33]:
# 시간 + 타석(또는 이닝) 방식

t1 = time.time()

################## 타자 ##################
for idx, hitter_column in enumerate(hitter_columns):
    
    tmp = make_average_variable(hitter_all2, YEAR, hitter_column, '타석', ALPHA, 'time_and_play_weighted')
    
    if idx == 0:
        hitter_time_and_play_weighted = tmp.copy()
    else:
        hitter_time_and_play_weighted = pd.concat([hitter_time_and_play_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(hitter_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
        
################## 투수 ##################
for idx, pitcher_column in enumerate(pitcher_columns):
    
    tmp = make_average_variable(pitcher_all2, YEAR, pitcher_column, '이닝', ALPHA, 'time_and_play_weighted')
    
    if idx == 0:
        pitcher_time_and_play_weighted = tmp.copy()
    else:
        pitcher_time_and_play_weighted = pd.concat([pitcher_time_and_play_weighted, tmp.iloc[:,[-1]]], axis=1)
        
    if (idx+1) % 10 == 0:
        print(f'{idx+1}/{len(pitcher_columns)}개 생성 완료!! == 경과시간: {round(time.time()-t1)}초')
        
hitter_time_and_play_weighted = data_fillna(hitter_time_and_play_weighted, YEAR)
pitcher_time_and_play_weighted = data_fillna(pitcher_time_and_play_weighted, YEAR)

10/57개 생성 완료!! == 경과시간: 68초
20/57개 생성 완료!! == 경과시간: 129초
30/57개 생성 완료!! == 경과시간: 196초
40/57개 생성 완료!! == 경과시간: 255초
50/57개 생성 완료!! == 경과시간: 314초
10/61개 생성 완료!! == 경과시간: 406초
20/61개 생성 완료!! == 경과시간: 460초
30/61개 생성 완료!! == 경과시간: 516초
40/61개 생성 완료!! == 경과시간: 573초
50/61개 생성 완료!! == 경과시간: 631초
60/61개 생성 완료!! == 경과시간: 688초


#### 8. FA/비FA를 나눈 후 Train/test Split 후 최종 저장

In [34]:
def fa_split(dataset):
    
    fa_ids = list(set(dataset.loc[dataset['FA여부'] == 1, 'ID']))
    nonfa_ids = list(set(dataset['ID']) - set(fa_ids))
    
    fa_dataset = dataset[dataset['ID'].isin(fa_ids)].reset_index(drop=True)
    nonfa_dataset = dataset[dataset['ID'].isin(nonfa_ids)].reset_index(drop=True)
    
    return fa_dataset, nonfa_dataset

###############################################################
###############################################################

def train_test_split(dataset):
    
    train = dataset[dataset['연도'] <= 2019].reset_index(drop=True)
    test = dataset[dataset['연도'] == 2020].reset_index(drop=True)
    
    return train, test

###############################################################
###############################################################

def fa_split_to_save(dataset):

    dataset_fa, dataset_nonfa = fa_split(dataset)
    dataset_fa_train, dataset_fa_test = train_test_split(dataset_fa)
    dataset_nonfa_train, dataset_nonfa_test = train_test_split(dataset_nonfa)
    
    return dataset_fa_train, dataset_fa_test, dataset_nonfa_train, dataset_nonfa_test

In [35]:
hitter_simple_fa_train, hitter_simple_fa_test, hitter_simple_nonfa_train, hitter_simple_nonfa_test = fa_split_to_save(hitter_simple)
pitcher_simple_fa_train, pitcher_simple_fa_test, pitcher_simple_nonfa_train, pitcher_simple_nonfa_test = fa_split_to_save(pitcher_simple)

hitter_simple_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_simple_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
hitter_simple_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_simple_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

pitcher_simple_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_simple_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
pitcher_simple_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_simple_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

In [36]:
hitter_time_weighted_fa_train, hitter_time_weighted_fa_test, hitter_time_weighted_nonfa_train, hitter_time_weighted_nonfa_test = fa_split_to_save(hitter_time_weighted)
pitcher_time_weighted_fa_train, pitcher_time_weighted_fa_test, pitcher_time_weighted_nonfa_train, pitcher_time_weighted_nonfa_test = fa_split_to_save(pitcher_time_weighted)

hitter_time_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_time_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
hitter_time_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_time_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

pitcher_time_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

In [37]:
hitter_play_weighted_fa_train, hitter_play_weighted_fa_test, hitter_play_weighted_nonfa_train, hitter_play_weighted_nonfa_test = fa_split_to_save(hitter_play_weighted)
pitcher_play_weighted_fa_train, pitcher_play_weighted_fa_test, pitcher_play_weighted_nonfa_train, pitcher_play_weighted_nonfa_test = fa_split_to_save(pitcher_play_weighted)

hitter_play_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_play_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
hitter_play_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_play_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

pitcher_play_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_play_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
pitcher_play_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_play_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

In [38]:
hitter_time_and_play_weighted_fa_train, hitter_time_and_play_weighted_fa_test, hitter_time_and_play_weighted_nonfa_train, hitter_time_and_play_weighted_nonfa_test = fa_split_to_save(hitter_time_and_play_weighted)
pitcher_time_and_play_weighted_fa_train, pitcher_time_and_play_weighted_fa_test, pitcher_time_and_play_weighted_nonfa_train, pitcher_time_and_play_weighted_nonfa_test = fa_split_to_save(pitcher_time_and_play_weighted)

hitter_time_and_play_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_time_and_play_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
hitter_time_and_play_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
hitter_time_and_play_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')

pitcher_time_and_play_weighted_fa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_fa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_and_play_weighted_fa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_fa_test.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_and_play_weighted_nonfa_train.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_nonfa_train.csv', index = False, encoding = 'utf-8-sig')
pitcher_time_and_play_weighted_nonfa_test.to_csv('선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_nonfa_test.csv', index = False, encoding = 'utf-8-sig')